In [1]:
import pandas as pd
import requests
!pip install bs4
from bs4 import BeautifulSoup
!pip install geocoder
import geocoder
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.



In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'html.parser')

In [3]:
text = soup.find_all('table', attrs={'class':'wikitable sortable'})

In [4]:
texttemp2 = text[0].find_all('tr')

In [5]:
len(texttemp2)

181

In [6]:
result = []

for temp in texttemp2:
    
    temp2= temp.find_all('td')
    
    if (temp2!=[] and temp2[1].text[0:-1]!= 'Not assigned'):
        
        postalcode= temp2[0].text[0:-1]
        borough= temp2[1].text[0:-1]
        neighborhood= temp2[2].text[0:-1]
        
        if (neighborhood== 'Not assigned'):
            
            neighborhood= borough
            
        result.append((postalcode, borough, neighborhood)) 

In [7]:
df = pd.DataFrame(result, columns=['Postalcode', 'Borough', 'Neighborhood'])
df

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [8]:
df.shape

(103, 3)

In [10]:
dftemp= pd.DataFrame(columns=['Latitude','Longitude'])

for data, spec in zip (df['Borough'], df['Neighborhood']):
    
    splitted_string =spec.split(',')

    address= data +', '+ splitted_string [0] +', TO'
    geolocator = Nominatim(user_agent="to_explorer")
    location = geolocator.geocode(address)
    
    if (location== None):
        address= data +', TO'
        geolocator = Nominatim(user_agent="to_explorer")        
        location = geolocator.geocode(address)        
    
    latitude = location.latitude
    longitude = location.longitude
        
    dftemp= dftemp.append({'Latitude': latitude,
                           'Longitude': longitude}, ignore_index=True)
dftemp    

,Latitude,Longitude
0,43.761224,-79.323986
1,43.732658,-79.311189
2,43.654174,-79.380812
3,43.722079,-79.437507
4,43.654174,-79.380812
...,...,...
98,43.647381,-79.511333
99,43.660568,-79.383785
100,43.721789,-79.374027
101,43.651301,-79.495475


In [11]:
df = pd.concat([df, dftemp], axis=1)
df

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.761224,-79.323986
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654174,-79.380812
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722079,-79.437507
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.654174,-79.380812
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.647381,-79.511333
99,M4Y,Downtown Toronto,Church and Wellesley,43.660568,-79.383785
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.721789,-79.374027
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.651301,-79.495475


In [12]:
df.shape

(103, 5)